# Assignment 01: [creative title name]

### Maxim Ryabinov (U02204083)
### CAP4641: Natural Language Processing 
### Instructor: Dr. Ankur Mali 
### University of South Florida (Spring 2025)

---

# 1. Initial Setup
- Sets random seeds to ensure reproducibility.
- Creates 


In [43]:
import os

# Suppress TensorFlow warnings.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import random
import datetime


tf.random.set_seed(1234)
np.random.seed(1234)
random.seed(1234)


# --------------------
# Data Storage Classes
# --------------------
class Hyperparameters():    
    def __init__(self, learning_rate=None, hidden_layers=None, hidden_sizes=None, batch_size=None, optimizer=None, activation_function=None):
        self.learning_rate = learning_rate
        self.hidden_layers = hidden_layers
        self.hidden_sizes = hidden_sizes
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.activation_function = activation_function
    
    def __str__(self):
        return (f"Learning Rate: {self.learning_rate:.4f} | Num Layers: {self.hidden_layers} | "
                f"Hidden Sizes: {self.hidden_sizes} | Batch Size: {self.batch_size} | "
                f"Optimizer: {self.optimizer.__name__} | Activation Function: {self.activation_function.__name__}\n")
    
    @staticmethod
    def get_hyperparameter_choices():
        # Returns a dictionary of possible hyperparameter values.
        return {
            "learning_rates": [0.001, 0.0005, 0.0001],
            "hidden_layers": [1, 2, 3],
            "hidden_sizes": [128, 256, 512],
            "batch_sizes": [32, 64, 128],
            "optimizers": [tf.keras.optimizers.Adam, tf.keras.optimizers.SGD, tf.keras.optimizers.RMSprop],
            "activation_functions": [tf.nn.relu, tf.nn.tanh, tf.nn.leaky_relu]
        }
        
    def sample_random(self):        
        choices = self.get_hyperparameter_choices()
        self.learning_rate = random.choice(choices["learning_rates"])
        self.hidden_layers = random.choice(choices["hidden_layers"])
        picked_size = random.choice(choices["hidden_sizes"])
        self.hidden_sizes = [picked_size] * self.hidden_layers
        self.batch_size = random.choice(choices["batch_sizes"])
        self.optimizer = random.choice(choices["optimizers"])
        self.activation_function = random.choice(choices["activation_functions"])
        
        

class PerformanceMetrics():
    def __init__(self, loss, accuracy, precision, recall):
        self.loss = loss
        self.accuracy = accuracy
        self.precision = precision
        self.recall = recall
    
    def __str__(self):
        return (f"Loss: {self.loss:.4f} | Accuracy: {self.accuracy:.4f} | "
                f"Precision: {self.precision:.4f} | Recall: {self.recall:.4f}\n")

class Result():
    def __init__(self, hyperparameters, performance_metrics):
        self.hyperparameters = hyperparameters
        self.performance_metrics = performance_metrics
        
    def __str__(self):
        return (f"Hyperparameters:\n"
                f"Learning Rate: {self.hyperparameters.learning_rate:.4f} | Num Layers: {self.hyperparameters.hidden_layers} | "
                f"Hidden Sizes: {self.hyperparameters.hidden_sizes} | Batch Size: {self.hyperparameters.batch_size} | "
                f"Optimizer: {self.hyperparameters.optimizer.__name__} | Activation Function: {self.hyperparameters.activation_function.__name__}\n"
                f"\nPerformance Metrics:\n"
                f"Loss: {self.performance_metrics.loss:.4f} | Accuracy: {self.performance_metrics.accuracy:.4f} | "
                f"Precision: {self.performance_metrics.precision:.4f} | Recall: {self.performance_metrics.recall:.4f}\n")


# ------------------------------------------
# Tokenizer and Preprocessing Util Functions
# ------------------------------------------
def create_tokenizer(texts, is_char_level, num_words=None):
    """
    Create and fit a character-level tokenizer.

    Args:
        texts (list of str): List of texts.
        num_words (int or None): Maximum number of tokens to keep.

    Returns:
        tokenizer: A fitted Tokenizer instance.
    """
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, char_level=is_char_level, lower=True)
    tokenizer.fit_on_texts(texts)
    return tokenizer

def texts_to_bow(tokenizer, texts):
    """
    Convert texts to a bag-of-characters representation.

    Args:
        tokenizer: A fitted character-level Tokenizer.
        texts (list of str): List of texts.

    Returns:
        Numpy array representing the binary bag-of-characters for each text.
    """
    # texts_to_matrix with mode 'binary' produces a fixed-length binary vector per text.
    matrix = tokenizer.texts_to_matrix(texts, mode='binary')
    return matrix

def one_hot_encode(labels, num_classes=2):
    """
    Convert numeric labels to one-hot encoded vectors.
    """
    return np.eye(num_classes)[labels]

# Hyper-Parameter Random Search
def sample_hyperparameters():
    """
    Generate random hyper-parameters for the model.
    """
    learning_rates = [0.001, 0.0005, 0.0001]
    hidden_layers = [1, 2, 3]
    hidden_sizes = [128, 256, 512]
    batch_sizes = [32, 64, 128]
    optimizers = [tf.keras.optimizers.Adam, tf.keras.optimizers.SGD, tf.keras.optimizers.RMSprop]
    activation_functions = [tf.nn.relu, tf.nn.tanh, tf.nn.leaky_relu]

    # Randomly sample one value from each category
    learning_rate = random.choice(learning_rates)
    num_hidden = random.choice(hidden_layers)
    picked_size = random.choice(hidden_sizes)
    hidden_sizes = [picked_size for i in range(num_hidden)]
    batch_size = random.choice(batch_sizes)
    optimizer = random.choice(optimizers)
    act_func = random.choice(activation_functions)

    return Hyperparameters(learning_rate, num_hidden, hidden_sizes, batch_size, optimizer, act_func)
    

# --------------------
# For logging purposes
# --------------------
def log_hyperparameters(learning_rate, num_layers, hidden_sizes, batch_size, optimizer, act_func, log_filename="training_logs.txt"):
    # Get the current timestamp to label the start of the run
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Prepare the log entry for the hyper-parameters
    log_entry = (f"\nHyper-parameters:\n"
                 f"Learning Rate: {learning_rate:.4f} | Num Layers: {num_layers} | Hidden Sizes: {hidden_sizes} | "
                 f"Batch Size: {batch_size} | Optimizer: {optimizer.__name__} | Activation Function: {act_func.__name__}\n")
    
    # Append to the log file
    with open(log_filename, 'a') as log_file:
        log_file.write(log_entry)

def log_training_info(epoch, epoch_loss, val_loss, accuracy, precision, recall, log_filename="training_logs.txt"):
    # Get the current timestamp to label the start of the run
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Prepare the log entry for each epoch
    log_entry = (f"\n[{timestamp}] - Epoch {epoch+1:02d} | Training Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | "
                 f"Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}\n")
    
    # Append to the log file
    with open(log_filename, 'a') as log_file:
        log_file.write(log_entry)

def log_final_evaluation(test_loss, test_accuracy, test_precision, test_recall, log_filename="training_logs.txt"):
    # Prepare the final evaluation log entry
    final_log_entry = (f"\nFinal Evaluation on Test Set:\n"
                       f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.4f} | "
                       f"Test Precision: {test_precision:.4f} | Test Recall: {test_recall:.4f}\n"
                       f"\n=======================================================================================================================\n")
    
    # Append final evaluation to the log file
    with open(log_filename, 'a') as log_file:
        log_file.write(final_log_entry)

# Defining the MLP Model

In [21]:
# --------------------
# MLP Class Definition
# --------------------
class MLP(object):
    def __init__(self, size_input, num_hidden, hidden_sizes, size_output, batch_size, optimizer, learning_rate, act_func, device=None):
        """
        size_input: int, size of input layer
        size_hidden1: int, size of the 1st hidden layer
        size_hidden2: int, size of the 2nd hidden layer
        size_hidden3: int, size of the 3rd hidden layer (not used in compute_output here)
        size_output: int, size of output layer
        device: str or None, either 'cpu' or 'gpu' or None.
        """
        self.size_input = size_input # Number of features
        self.num_hidden = num_hidden # Number of hidden layers
        self.hidden_sizes = hidden_sizes # List of hidden layer sizes
        self.size_output = size_output # Number of classes (Binary Classification)
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.optimizer = optimizer # Optimizer class (e.g. Adam, SGD, RMSprop)
        self.act_func = act_func # Activation function to use (e.g. ReLU, Tanh, Leaky ReLU)
        self.device = device # Device to run the model on (cpu or gpu)
        
        self.hidden_layers = []
        
        w = tf.Variable(tf.random.normal([self.size_input, self.hidden_sizes[0]], stddev=0.1))
        b = tf.Variable(tf.zeros([1, self.hidden_sizes[0]]))
        self.hidden_layers.append((w, b))
        
        for i in range(1, self.num_hidden):
            w = tf.Variable(tf.random.normal([self.hidden_sizes[i-1], self.hidden_sizes[i]], stddev=0.1))
            b = tf.Variable(tf.zeros([1, self.hidden_sizes[i]]))
            self.hidden_layers.append((w, b))
        
        w = tf.Variable(tf.random.normal([self.hidden_sizes[-1], self.size_output], stddev=0.1))
        b = tf.Variable(tf.zeros([1, self.size_output]))
        self.hidden_layers.append((w, b))

    def forward(self, X):
        """
        Forward pass.
        X: Tensor, inputs.
        """
        if self.device is not None:
            with tf.device('gpu:0' if self.device == 'gpu' else 'cpu'):
                self.y = self.compute_output(X)
        else:
            self.y = self.compute_output(X)
        return self.y

    def loss(self, y_pred, y_true):
        """
        Computes the loss between predicted and true outputs.
        y_pred: Tensor of shape (batch_size, size_output)
        y_true: Tensor of shape (batch_size, size_output)
        """
        y_true_tf = tf.cast(y_true, dtype=tf.float32)
        y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
        cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        loss_x = cce(y_true_tf, y_pred_tf)
        return loss_x

    def backward(self, X_train, y_train):
        """
        Backward pass: compute and apply gradients of the loss with respect to the variables.
        """
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train)
            current_loss = self.loss(predicted, y_train)
        
        trainable_vars = [var for w, b in self.hidden_layers for var in (w, b)]    
        
        grads = tape.gradient(current_loss, trainable_vars)
        
        optimizer = self.optimizer(learning_rate=self.learning_rate)
        optimizer.apply_gradients(zip(grads, trainable_vars))

    def compute_output(self, X):
        """
        Custom method to compute the output tensor during the forward pass.
        """        
        # Cast X to float32
        X_tf = tf.cast(X, dtype=tf.float32)
        z = X_tf
        
        # Goes through each hidden layer and applies the weights and biases
        for w, b in self.hidden_layers:
            z = tf.matmul(z, w) + b
            
            # If the current layer is the last layer, then its the output layer.
            if (w, b) == self.hidden_layers[-1]:
                return z
            
            z = self.act_func(z)

# Training the Model (Train Loop)

In [22]:
def train_model(model, X_train, X_val, y_train, y_val, epochs = 3):
    num_batches = int(np.ceil(X_train.shape[0] / model.batch_size))

    print("\nStarting training...\n")
    for epoch in range(epochs):
        # Shuffle training data at the start of each epoch.
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        X_train = X_train[indices]
        y_train = y_train[indices]

        epoch_loss = 0
        for i in range(num_batches):
            start = i * model.batch_size
            end = min((i+1) * model.batch_size, X_train.shape[0])
            X_batch = X_train[start:end]
            y_batch = y_train[start:end]

            # Compute gradients and update weights.
            predictions = model.forward(X_batch)
            loss_value = model.loss(predictions, y_batch)
            model.backward(X_batch, y_batch)

            epoch_loss += loss_value.numpy() * (end - start)

        epoch_loss /= X_train.shape[0]

        # Evaluate on validation set.
        val_logits = model.forward(X_val)
        val_loss = model.loss(val_logits, y_val).numpy()
        val_preds = np.argmax(val_logits.numpy(), axis=1)
        true_val = np.argmax(y_val, axis=1)
        accuracy = np.mean(val_preds == true_val)
        precision = precision_score(true_val, val_preds)
        recall = recall_score(true_val, val_preds)

        print(f"Epoch {epoch+1:02d} | Training Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | "
            f"Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

# Evaluation on Test Set (Test Loop)

In [40]:
def test_model(model, X_test, y_test):
    # -------------------------------
    # Final Evaluation on Test Set
    # -------------------------------
    print("\nEvaluating on test set...")
    test_logits = model.forward(X_test)
    test_loss = model.loss(test_logits, y_test).numpy()
    test_preds = np.argmax(test_logits.numpy(), axis=1)
    true_test = np.argmax(y_test, axis=1)
    test_accuracy = np.mean(test_preds == true_test)
    test_precision = precision_score(true_test, test_preds)
    test_recall = recall_score(true_test, test_preds)

    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.4f} | "
        f"Test Precision: {test_precision:.4f} | Test Recall: {test_recall:.4f}")
    
    return test_loss, test_accuracy, test_precision, test_recall

# Loading IMDB Dataset and Tokenizing

In [7]:
# -------------------------------
# Load and Prepare the IMDB Dataset
# -------------------------------
print("Loading IMDB dataset...\n")
# Load the IMDB reviews dataset with the 'as_supervised' flag so that we get (text, label) pairs.
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews',
                                           split=['train', 'test'],
                                           as_supervised=True,
                                           with_info=True)

# Convert training dataset to lists.
train_texts = []
train_labels = []
for text, label in tfds.as_numpy(ds_train):
    # Decode byte strings to utf-8 strings.
    train_texts.append(text.decode('utf-8'))
    train_labels.append(label)
train_labels = np.array(train_labels)

# Create a validation set from the training data (20% for validation).
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42)

# Convert test dataset to lists.
test_texts = []
test_labels = []
for text, label in tfds.as_numpy(ds_test):
    test_texts.append(text.decode('utf-8'))
    test_labels.append(label)
test_labels = np.array(test_labels)

print("IMBD Dataset loaded:")
print(f"Train samples: {len(train_texts)}, Validation samples: {len(val_texts)}, Test samples: {len(test_texts)}\n")


# -------------------------------
# Preprocessing: Tokenization and Vectorization
# -------------------------------
print("Tokenizing texts...")
# Build the character-level tokenizer on the training texts.
char_tokenizer = create_tokenizer(train_texts, is_char_level=True)
print("Char tokenizer vocabulary size:", len(char_tokenizer.word_index) + 1)

word_tokenizer = create_tokenizer(train_texts, is_char_level=False, num_words=10000) # Limited to the top 10,000 common words.
print("Word tokenizer vocabulary size:", len(word_tokenizer.word_index) + 1, "\n")

print("Converting texts to bag-of-characters...")
# Convert texts to bag-of-characters representation.
X_train_char = texts_to_bow(char_tokenizer, train_texts)
X_val_char   = texts_to_bow(char_tokenizer, val_texts)
X_test_char  = texts_to_bow(char_tokenizer, test_texts)

print("Converting texts to bag-of-words...")
# Convert texts to bag-of-words representation.
X_train_word = texts_to_bow(word_tokenizer, train_texts)
X_val_word   = texts_to_bow(word_tokenizer, val_texts)
X_test_word  = texts_to_bow(word_tokenizer, test_texts)

print("Performing one-hot encoding on labels...\n")
# Convert labels to one-hot encoding.
y_train = one_hot_encode(train_labels)
y_val   = one_hot_encode(val_labels)
y_test  = one_hot_encode(test_labels)
print("Preprocessing complete.")


Loading IMDB dataset...

IMBD Dataset loaded:
Train samples: 20000, Validation samples: 5000, Test samples: 25000

Tokenizing texts...
Char tokenizer vocabulary size: 134
Word tokenizer vocabulary size: 80169 

Converting texts to bag-of-characters...
Converting texts to bag-of-words...
Performing one-hot encoding of labels...

Preprocessing complete.


# Character Tokenization Vs. Word Tokenization


In [19]:
# -------------------------------
# Model Setup
# -------------------------------
# The input size is determined by the dimension of the bag-of-characters vector.
char_size_input = X_train_char.shape[1]
word_size_input = X_train_word.shape[1]

# Parameters used for both models.
num_hidden = 3 # Number of hidden layers
hidden_sizes = [128, 64, 32] # Size of each hidden layer
batch_size = 128
optimizer = tf.keras.optimizers.Adam
learning_rate = 0.001
act_func = tf.nn.relu # Activation function used
size_output  = 2 # Binary classification

char_model = MLP(size_input=char_size_input,
            num_hidden=num_hidden,
            hidden_sizes=hidden_sizes,
            size_output=size_output,
            batch_size=batch_size,
            optimizer=optimizer,
            learning_rate=learning_rate,
            act_func=act_func,
            device=None)

word_model = MLP(size_input=word_size_input,
            num_hidden=num_hidden,
            hidden_sizes=hidden_sizes,
            size_output=size_output,
            batch_size=batch_size,
            optimizer=optimizer,
            learning_rate=learning_rate,
            act_func=act_func,
            device=None)

# -------------------------------
# Training and Evaluation (Testing)
# -------------------------------

print("\nTraining the character-level model...")
train_model(char_model, X_train_char, X_val_char, y_train, y_val)
test_model(char_model, X_test_char, y_test)

print("\n-----------------------------------------------------------------------------------------------------------------------")

print("\nTraining the word-level model...")
train_model(word_model, X_train_word, X_val_word, y_train, y_val)
test_model(word_model, X_test_word, y_test)


Training the character-level model...

Starting training...

Epoch 01 | Training Loss: 0.6782 | Val Loss: 0.6639 | Accuracy: 0.6074 | Precision: 0.5954 | Recall: 0.5932
Epoch 02 | Training Loss: 0.6695 | Val Loss: 0.6702 | Accuracy: 0.5876 | Precision: 0.6241 | Recall: 0.3754
Epoch 03 | Training Loss: 0.6653 | Val Loss: 0.6780 | Accuracy: 0.5886 | Precision: 0.5529 | Recall: 0.7917

Evaluating on test set...
Test Loss: 0.6725 | Test Accuracy: 0.5978 | Test Precision: 0.5692 | Test Recall: 0.8040

-----------------------------------------------------------------------------------------------------------------------

Training the word-level model...

Starting training...

Epoch 01 | Training Loss: 0.4296 | Val Loss: 0.3437 | Accuracy: 0.8642 | Precision: 0.8115 | Recall: 0.9377
Epoch 02 | Training Loss: 0.2606 | Val Loss: 0.2990 | Accuracy: 0.8834 | Precision: 0.8478 | Recall: 0.9257
Epoch 03 | Training Loss: 0.2244 | Val Loss: 0.3011 | Accuracy: 0.8900 | Precision: 0.8751 | Recall: 0.9

# Hyper_parameter Optimization

In [44]:
tf.random.set_seed(1234)
np.random.seed(1234)
random.seed(1234)

# -------------------------------
# Model Setup
# -------------------------------
# The input size is determined by the dimension of the bag-of-characters vector.
size_input = X_train_word.shape[1]
size_output  = 2 # Binary Classification
results = []





params = Hyperparameters()
params.sample_random()
print(params)


model = MLP(size_input = size_input,
            num_hidden = params.hidden_layers,
            hidden_sizes = params.hidden_sizes,
            size_output = size_output,
            batch_size = params.batch_size,
            optimizer = params.optimizer,
            learning_rate = params.learning_rate,
            act_func = params.activation_function,
            device=None)


train_model(model, X_train_word, X_val_word, y_train, y_val, epochs=1)
pm = PerformanceMetrics(*test_model(model, X_test_word, y_test))
results.append(Result(params, pm))

print("\n-----------------------------------------------------------------------------------------------------------------------")
for result in results:
    print(result)




# model = MLP(size_input=size_input,
#             num_hidden=num_hidden,
#             hidden_sizes=hidden_sizes,
#             size_output=size_output,
#             batch_size=batch_size,
#             optimizer=optimizer,
#             learning_rate=learning_rate,
#             act_func=act_func,
#             device=None)

# -------------------------------
# Training Parameters and Loop
# -------------------------------
# batch_size = 128
# for i in range(3):
#     epochs = 3
#     num_batches = int(np.ceil(X_train_word.shape[0] / model.batch_size))
    
#     log_hyperparameters(learning_rate, num_hidden, hidden_sizes, batch_size, optimizer, act_func)

#     print("\nStarting training...\n")
#     for epoch in range(epochs):
#         # Shuffle training data at the start of each epoch.
#         indices = np.arange(X_train_word.shape[0])
#         np.random.shuffle(indices)
#         X_train_word = X_train_word[indices]
#         y_train = y_train[indices]

#         epoch_loss = 0
#         for i in range(num_batches):
#             start = i * model.batch_size
#             end = min((i+1) * model.batch_size, X_train_word.shape[0])
#             X_batch = X_train_word[start:end]
#             y_batch = y_train[start:end]

#             # Compute gradients and update weights.
#             predictions = model.forward(X_batch)
#             loss_value = model.loss(predictions, y_batch)
#             model.backward(X_batch, y_batch)

#             epoch_loss += loss_value.numpy() * (end - start)

#         epoch_loss /= X_train_word.shape[0]

#         # Evaluate on validation set.
#         val_logits = model.forward(X_val_char)
#         val_loss = model.loss(val_logits, y_val).numpy()
#         val_preds = np.argmax(val_logits.numpy(), axis=1)
#         true_val = np.argmax(y_val, axis=1)
#         accuracy = np.mean(val_preds == true_val)
#         precision = precision_score(true_val, val_preds)
#         recall = recall_score(true_val, val_preds)

#         print(f"Epoch {epoch+1:02d} | Training Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | "
#             f"Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
        
#         log_training_info(epoch, epoch_loss, val_loss, accuracy, precision, recall)

#     # -------------------------------
#     # Final Evaluation on Test Set
#     # -------------------------------
#     print("\nEvaluating on test set...")
#     test_logits = model.forward(X_test_char)
#     test_loss = model.loss(test_logits, y_test).numpy()
#     test_preds = np.argmax(test_logits.numpy(), axis=1)
#     true_test = np.argmax(y_test, axis=1)
#     test_accuracy = np.mean(test_preds == true_test)
#     test_precision = precision_score(true_test, test_preds)
#     test_recall = recall_score(true_test, test_preds)

#     print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.4f} | "
#         f"Test Precision: {test_precision:.4f} | Test Recall: {test_recall:.4f}")

#     log_final_evaluation(test_loss, test_accuracy, test_precision, test_recall)

Learning Rate: 0.0005 | Num Layers: 1 | Hidden Sizes: [128] | Batch Size: 32 | Optimizer: RMSprop | Activation Function: relu


Starting training...

Epoch 01 | Training Loss: 0.3790 | Val Loss: 0.3205 | Accuracy: 0.8734 | Precision: 0.8651 | Recall: 0.8754

Evaluating on test set...
Test Loss: 0.3148 | Test Accuracy: 0.8759 | Test Precision: 0.8780 | Test Recall: 0.8732

-----------------------------------------------------------------------------------------------------------------------
Hyperparameters:
Learning Rate: 0.0005 | Num Layers: 1 | Hidden Sizes: [128] | Batch Size: 32 | Optimizer: RMSprop | Activation Function: relu
Performance Metrics:
Loss: 0.3148 | Accuracy: 0.8759 | Precision: 0.8780 | Recall: 0.8732

